# Rate-Lock Fallout Prediction with **PySpark**

## Business Problem
Borrowers lock an interest rate for 30–90 days while a mortgage loan goes through underwriting. If the loan **falls out** (does not fund before the lock expires), the lender loses revenue and incurs hedge/best-efforts penalties and wasted operations capacity.

**Goal:** Using **PySpark**, predict the probability that a rate-locked loan will **fall out** so teams can prioritize pricing concessions, lock-extension budgets, and document-collection workflows.

### What this notebook includes
1. Environment setup and Spark session
2. Synthetic data generation: **locks**, **loans**, **daily market rates**, **pipeline events**
3. Labeling loans as **fell_out** based on lock expiry
4. Feature engineering (latencies, rate gap, credit/capacity, ops friction)
5. Time-aware train/test split to avoid leakage
6. Modeling with **Logistic Regression** and **GBTClassifier** (MLlib)
7. Evaluation (AUC-ROC, AUC-PR) and **risk buckets** (LOW/MEDIUM/HIGH)
8. Example exports of high-risk loans

> 💡 **Why PySpark?** Joins + window features over tens of millions of rows, time-based splits, training at scale, and optional Structured Streaming for near-real-time pipeline scoring.


## 1) Environment Setup
Run this cell on Google Colab or any fresh environment.

> If you already have Spark locally, you can skip the install. This notebook targets Spark **3.5.1**.

In [1]:
# If needed, install PySpark
!pip install -q pyspark==3.5.1


## 2) Create Spark Session
We start in local mode for demo purposes. In production, switch `.master()` to your cluster manager (YARN, Kubernetes).

In [2]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .appName("RateLockFalloutPrediction")
    .master("local[*]")
    .config("spark.sql.shuffle.partitions", "8")
    .getOrCreate()
)
spark


## 3) Synthetic Data Generation
We create 4 realistic tables:

- **`rates_daily`**: daily market 30-year rate timeline
- **`loans`**: borrower/loan features (LTV, DTI, FICO, purpose, occupancy, state) and ops friction counters
- **`locks`**: one lock per loan with lock date, term, and lock rate
- **`pipeline_events`**: milestone timestamps (`disclosures_out`, `docs_received`, `appraisal_in`, `conditions_submitted`, `clear_to_close`, `funded`)

The generator purposefully creates a mix of funded vs. non-funded loans and latency patterns to enable a predictive signal.

In [3]:
from pyspark.sql import functions as F, types as T
from datetime import date, timedelta
import random

spark.conf.set("spark.sql.session.timeZone", "UTC")

random.seed(7)
N_LOANS = 25000  # Adjust down if your machine is resource constrained

# --- 3.1 Daily market rates (2022-01-01 to 2025-08-01) ---
start_rates = date(2022,1,1)
end_rates   = date(2025,8,1)
days = (end_rates - start_rates).days
base = 3.0
rows_rates = []
level = base
for d in range(days+1):
    dt = start_rates + timedelta(days=d)
    # Random walk with slight upward drift until 2023, then oscillate
    drift = 0.0005 if dt < date(2023,12,31) else -0.0002
    shock = random.uniform(-0.03, 0.03)
    level = max(2.0, min(9.0, level + drift + shock))
    rows_rates.append((dt, round(level, 3)))

rates_schema = T.StructType([
    T.StructField("as_of_date", T.DateType(), False),
    T.StructField("market_rate_30yr", T.DoubleType(), False)
])
rates = spark.createDataFrame(rows_rates, schema=rates_schema)
rates.cache(); print("rates:", rates.count()); rates.orderBy("as_of_date").show(3)

# --- 3.2 Loans ---
states   = ["CA","TX","FL","NY","WA","IL","PA","OH","GA","NC"]
purpose  = ["purchase","refi","cashout_refi"]
occ      = ["primary","second_home","investment"]

def sample_fico():
    return int(max(520, min(840, random.gauss(705, 60))))

def sample_ltv():
    return round(min(0.97, max(0.5, random.gauss(0.78, 0.10))), 3)

def sample_dti():
    return round(min(0.55, max(0.15, random.gauss(0.34, 0.10))), 3)

loans_rows = [
    (
        i,                      # loan_id
        sample_ltv(),           # LTV
        sample_dti(),           # DTI
        sample_fico(),          # FICO
        random.choice(purpose), # purpose
        random.choice(occ),     # occupancy
        random.choice(states),  # property_state
        random.randint(0,5),    # missing_doc_count
        random.randint(0,3),    # resubmissions
        random.randint(1,8),    # touches
    )
    for i in range(1, N_LOANS+1)
]
loans_schema = T.StructType([
    T.StructField("loan_id", T.IntegerType(), False),
    T.StructField("LTV", T.DoubleType(), True),
    T.StructField("DTI", T.DoubleType(), True),
    T.StructField("FICO", T.IntegerType(), True),
    T.StructField("purpose", T.StringType(), True),
    T.StructField("occupancy", T.StringType(), True),
    T.StructField("property_state", T.StringType(), True),
    T.StructField("missing_doc_count", T.IntegerType(), True),
    T.StructField("resubmissions", T.IntegerType(), True),
    T.StructField("touches", T.IntegerType(), True),
])
loans = spark.createDataFrame(loans_rows, schema=loans_schema)
loans.cache(); print("loans:", loans.count()); loans.show(3, truncate=False)

# --- 3.3 Locks (one per loan) ---
lock_terms = [30,45,60]
channels   = ["retail","broker","correspondent"]
lock_rows = []
for i in range(1, N_LOANS+1):
    lock_date = start_rates + timedelta(days=random.randint(0, days))
    term = random.choices(lock_terms, weights=[0.5,0.35,0.15])[0]
    # Approx market on lock date (we'll still join on canonical rates below)
    market = rows_rates[(lock_date - start_rates).days][1]
    # Locked rate around market with some spread
    lock_rate = round(market + random.uniform(-0.5, 0.7), 3)
    points = round(max(0.0, random.gauss(0.25, 0.20)), 3)
    channel = random.choice(channels)
    lock_rows.append((i, i, lock_date, term, lock_rate, points, channel))

locks_schema = T.StructType([
    T.StructField("lock_id", T.IntegerType(), False),
    T.StructField("loan_id", T.IntegerType(), False),
    T.StructField("lock_date", T.DateType(), False),
    T.StructField("lock_term_days", T.IntegerType(), True),
    T.StructField("lock_rate", T.DoubleType(), True),
    T.StructField("points", T.DoubleType(), True),
    T.StructField("channel", T.StringType(), True),
])
locks = spark.createDataFrame(lock_rows, schema=locks_schema)
locks.cache(); print("locks:", locks.count()); locks.show(3, truncate=False)

# --- 3.4 Pipeline events (milestones) ---
event_types = ["disclosures_out","docs_received","appraisal_in","conditions_submitted","clear_to_close","funded"]
events_rows = []
for i in range(1, N_LOANS+1):
    lock_date = lock_rows[i-1][2]
    term      = lock_rows[i-1][3]
    # Decide whether this loan funds
    p_fund = 0.78  # overall funding rate
    funds = random.random() < p_fund
    # milestone timings relative to lock_date
    disclosures = lock_date + timedelta(days=random.randint(0, 3))
    docs       = disclosures + timedelta(days=random.randint(1, 12))
    appraisal  = docs + timedelta(days=random.randint(2, 14))
    conds      = appraisal + timedelta(days=random.randint(3, 10))
    ctc        = conds + timedelta(days=random.randint(2, 7))
    funded     = None
    if funds:
        jitter = random.randint(-5, 10)  # some fund before/near/after expiry
        funded_candidate = lock_date + timedelta(days=term + jitter)
        funded = funded_candidate
    # Record events
    events_rows.append((i, "disclosures_out", disclosures))
    events_rows.append((i, "docs_received",   docs))
    events_rows.append((i, "appraisal_in",     appraisal))
    events_rows.append((i, "conditions_submitted", conds))
    events_rows.append((i, "clear_to_close",   ctc))
    if funded:
        events_rows.append((i, "funded", funded))

events_schema = T.StructType([
    T.StructField("loan_id", T.IntegerType(), False),
    T.StructField("event_type", T.StringType(), False),
    T.StructField("event_ts", T.DateType(), False)
])
pipeline_events = spark.createDataFrame(events_rows, schema=events_schema)
pipeline_events.cache(); print("pipeline_events:", pipeline_events.count()); pipeline_events.show(6, truncate=False)


rates: 1309
+----------+----------------+
|as_of_date|market_rate_30yr|
+----------+----------------+
|2022-01-01|            2.99|
|2022-01-02|           2.969|
|2022-01-03|           2.979|
+----------+----------------+
only showing top 3 rows

loans: 25000
+-------+-----+-----+----+------------+-----------+--------------+-----------------+-------------+-------+
|loan_id|LTV  |DTI  |FICO|purpose     |occupancy  |property_state|missing_doc_count|resubmissions|touches|
+-------+-----+-----+----+------------+-----------+--------------+-----------------+-------------+-------+
|1      |0.881|0.55 |701 |cashout_refi|primary    |NY            |3                |1            |3      |
|2      |0.734|0.385|611 |refi        |second_home|NY            |0                |0            |5      |
|3      |0.807|0.435|764 |refi        |investment |NY            |4                |2            |8      |
+-------+-----+-----+----+------------+-----------+--------------+-----------------+-------------+

## 4) Unify & Label `fell_out`

A loan is labeled **`fell_out = 1`** if it has **no funding date** or the **funding date is after** `lock_date + lock_term_days + grace_days`.

In [4]:
from pyspark.sql import functions as F

# Pivot pipeline events to get one row per loan with milestone dates
pivoted = (
    pipeline_events
    .groupBy("loan_id")
    .pivot("event_type", ["disclosures_out","docs_received","appraisal_in","conditions_submitted","clear_to_close","funded"])
    .agg(F.min("event_ts"))  # earliest occurrence
)

# Attach market rate on lock date to compute rate_gap later
locks_enriched = (
    locks.join(
        rates.withColumnRenamed("as_of_date","lock_date"),
        on="lock_date", how="left"
    ).withColumnRenamed("market_rate_30yr","market_rate_on_lock")
)

base = (
    locks_enriched
    .join(loans, on="loan_id", how="left")
    .join(pivoted, on="loan_id", how="left")
    .withColumn("lock_expiry_date", F.expr("date_add(lock_date, lock_term_days)"))
    .withColumn("grace_days", F.lit(5))
)

labeled = base.withColumn(
    "fell_out",
    F.when(
        (F.col("funded").isNull()) | (F.col("funded") > F.expr("date_add(lock_expiry_date, grace_days)")),
        F.lit(1)
    ).otherwise(F.lit(0))
)

labeled.select("loan_id","lock_date","lock_term_days","funded","fell_out").show(10, truncate=False)
labeled.groupBy("fell_out").count().show()


+-------+----------+--------------+----------+--------+
|loan_id|lock_date |lock_term_days|funded    |fell_out|
+-------+----------+--------------+----------+--------+
|1      |2024-09-30|30            |2024-10-31|0       |
|2      |2022-09-28|60            |2022-11-24|0       |
|3      |2023-08-22|60            |NULL      |1       |
|4      |2025-02-05|45            |2025-03-18|0       |
|5      |2023-01-12|30            |NULL      |1       |
|6      |2025-08-01|30            |2025-09-07|1       |
|7      |2022-06-25|30            |2022-07-25|0       |
|8      |2023-01-25|30            |2023-02-26|0       |
|9      |2022-12-16|30            |2023-01-22|1       |
|10     |2023-12-06|60            |2024-01-30|0       |
+-------+----------+--------------+----------+--------+
only showing top 10 rows

+--------+-----+
|fell_out|count|
+--------+-----+
|       1|11591|
|       0|13409|
+--------+-----+



## 5) Feature Engineering

Features include:
- **Latencies**: `lock_to_docs_days`, `lock_to_appraisal_days`, `lock_to_ctc_days`
- **Rate gap**: `rate_gap = market_rate_on_lock - lock_rate`
- **Credit & capacity**: `FICO`, `LTV`, `DTI`
- **Ops friction**: `missing_doc_count`, `resubmissions`, `touches`
- **Categoricals**: `channel`, `purpose`, `occupancy`, `property_state`, `fico_band`

In [5]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml import Pipeline

feat = (
    labeled
    .withColumn("lock_to_docs_days", F.datediff("docs_received", "lock_date"))
    .withColumn("lock_to_appraisal_days", F.datediff("appraisal_in", "lock_date"))
    .withColumn("lock_to_ctc_days", F.datediff("clear_to_close", "lock_date"))
    .withColumn("rate_gap", F.col("market_rate_on_lock") - F.col("lock_rate"))
    .withColumn("fico_band",
        F.when(F.col("FICO") < 640, "subprime")
         .when(F.col("FICO") < 700, "nearprime")
         .when(F.col("FICO") < 740, "prime")
         .otherwise("superprime")
    )
)

label_col = "fell_out"
cat_cols = ["channel","purpose","occupancy","property_state","fico_band"]
num_cols = [
    "LTV","DTI","FICO","lock_term_days","rate_gap",
    "lock_to_docs_days","lock_to_appraisal_days","lock_to_ctc_days",
    "points","missing_doc_count","resubmissions","touches"
]

# Fill nulls for numeric columns
for c in num_cols:
    feat = feat.withColumn(c, F.coalesce(F.col(c), F.lit(0.0)))

indexers = [StringIndexer(inputCol=c, outputCol=f"{c}_idx", handleInvalid="keep") for c in cat_cols]
ohe = OneHotEncoder(
    inputCols=[f"{c}_idx" for c in cat_cols],
    outputCols=[f"{c}_oh" for c in cat_cols]
)
assembler = VectorAssembler(
    inputCols=num_cols + [f"{c}_oh" for c in cat_cols],
    outputCol="features"
)

prep = Pipeline(stages=[*indexers, ohe, assembler]).fit(feat)
data_ml = prep.transform(feat).select("features", label_col, "loan_id", "lock_date")
data_ml.show(3, truncate=False)


+------------------------------------------------------------------------------------------------------------------------------------------------+--------+-------+----------+
|features                                                                                                                                        |fell_out|loan_id|lock_date |
+------------------------------------------------------------------------------------------------------------------------------------------------+--------+-------+----------+
|(35,[0,1,2,3,4,5,6,7,8,9,10,11,12,17,18,27,33],[0.881,0.55,701.0,30.0,-0.6149999999999998,11.0,25.0,35.0,0.646,3.0,1.0,3.0,1.0,1.0,1.0,1.0,1.0])|0       |1      |2024-09-30|
|(35,[0,1,2,3,4,5,6,7,8,11,12,16,20,27,34],[0.734,0.385,611.0,60.0,-0.33000000000000007,11.0,18.0,33.0,0.495,5.0,1.0,1.0,1.0,1.0,1.0])           |0       |2      |2022-09-28|
|(35,[0,1,2,3,4,5,6,7,8,9,10,11,14,16,19,27,32],[0.807,0.435,764.0,60.0,0.2120000000000002,4.0,11.0,24.0,0.534,4.0,2.0,8.0,1.

## 6) Time-Aware Train/Test Split
To prevent look-ahead bias, we split by **`lock_date`**: train on older locks, test on newer.

In [6]:
cutoff = data_ml.agg(F.max("lock_date")).first()[0]
train = data_ml.where(F.expr(f"lock_date <= date_sub('{cutoff}', 60)"))
test  = data_ml.where(F.expr(f"lock_date  > date_sub('{cutoff}', 60)"))
print("Train rows:", train.count(), " Test rows:", test.count())

# Class weights for imbalance
pos = train.filter(F.col(label_col)==1).count()
neg = train.filter(F.col(label_col)==0).count()
beta = neg / max(1, pos)
train = train.withColumn("weight", F.when(F.col(label_col)==1, F.lit(beta)).otherwise(F.lit(1.0)))


Train rows: 23867  Test rows: 1133


## 7) Modeling (MLlib)
We compare a **Logistic Regression** baseline (interpretable) and a **GBTClassifier** (captures non-linearities).

In [7]:
from pyspark.ml.classification import LogisticRegression, GBTClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml import Pipeline

lr  = LogisticRegression(featuresCol="features", labelCol=label_col, weightCol="weight", maxIter=30, regParam=0.05)
gbt = GBTClassifier(featuresCol="features", labelCol=label_col, maxDepth=6, maxIter=60, stepSize=0.1, subsamplingRate=0.8)

lr_model  = lr.fit(train)
gbt_model = gbt.fit(train.drop("weight"))  # GBT doesn't use weightCol directly

lr_pred  = lr_model.transform(test)
gbt_pred = gbt_model.transform(test)

auc = BinaryClassificationEvaluator(labelCol=label_col, rawPredictionCol="rawPrediction", metricName="areaUnderROC")
aupr= BinaryClassificationEvaluator(labelCol=label_col, rawPredictionCol="rawPrediction", metricName="areaUnderPR")

print("LR   AUC:", round(auc.evaluate(lr_pred),  4), " AUPR:", round(aupr.evaluate(lr_pred),  4))
print("GBT  AUC:", round(auc.evaluate(gbt_pred), 4), " AUPR:", round(aupr.evaluate(gbt_pred), 4))


LR   AUC: 0.5071  AUPR: 0.5048
GBT  AUC: 0.5335  AUPR: 0.5066


## 8) Risk Buckets & Actionable Output
We convert probabilities into **LOW/MEDIUM/HIGH** fallout risk. Thresholds can be tuned to your cost/benefit curve.

In [12]:
from pyspark.sql.functions import col, when, regexp_extract

# Convert the probability vector to string and extract the second probability value
scored = gbt_pred.withColumn("prob_str", col("probability").cast("string")) \
                 .withColumn("p_fallout",
                             regexp_extract(col("prob_str"), r'\[.*?\,(.*?)\]', 1).cast("double")) \
    .withColumn("risk_bucket",
        when(col("p_fallout") >= 0.60, "HIGH")
        .when(col("p_fallout") >= 0.35, "MEDIUM")
        .otherwise("LOW")
    )

scored.groupBy("risk_bucket").count().show()

high = scored.where(col("risk_bucket")=="HIGH") \
            .select("loan_id","p_fallout","risk_bucket")
high.show(10, truncate=False)

+-----------+-----+
|risk_bucket|count|
+-----------+-----+
|     MEDIUM|  885|
|        LOW|  132|
|       HIGH|  116|
+-----------+-----+

+-------+------------------+-----------+
|loan_id|p_fallout         |risk_bucket|
+-------+------------------+-----------+
|372    |0.6299877695445567|HIGH       |
|749    |0.6721585736238243|HIGH       |
|1450   |0.7130389984985641|HIGH       |
|1910   |0.7163316876685629|HIGH       |
|2133   |0.6099559290958729|HIGH       |
|2198   |0.653823403765353 |HIGH       |
|2454   |0.6780412274650734|HIGH       |
|3023   |0.870663750066997 |HIGH       |
|3311   |0.6295178347779938|HIGH       |
|3316   |0.6214794432383939|HIGH       |
+-------+------------------+-----------+
only showing top 10 rows



## 9) Export Examples
In production, write to Parquet/Delta/DB; for demo we just show how to save. **Uncomment paths that suit your environment.**

In [14]:
# Example writes (adjust paths & permissions):


scored.write.mode("overwrite").parquet("/content/sample_data/data/")
high.write.mode("overwrite").csv("/content/sample_data/data", header=True)

# scored.write.mode("overwrite").parquet("/mnt/datalake/fallout_predictions/")
# high.write.mode("overwrite").csv("/mnt/datalake/fallout_high/", header=True)


## 10) Notes, Validation & Next Steps
- **Leakage control**: Keep features that would be known **before** funding when scoring live loans.
- **Imbalance**: Tune thresholds, use cost-sensitive metrics (expected value), and monitor precision/recall by channel.
- **Calibration**: Apply Platt/Isotonic calibration for budgetable probabilities.
- **Monitoring**: Track drift of `rate_gap`, FICO distribution, channel mix; retrain on rolling windows.
- **Operations integration**: Feed HIGH-risk loans to pricing/ops playbooks (doc chase, extension budgeting, LO callbacks).